# Practice Session 1: Policy and Value Iteration

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/124XxIn0ML8XslcPs4Cnwimab-EuFW8D6#scrollTo=Hr2tPjlrf-rP)

## Setting up the RL task with Gymnasium

Gymnasium (https://gymnasium.farama.org/) is a standard interface for simulating Markov Decision Processes (MDPs) based on OpenAI's Gym library.

An MDP is a modelling tool for Reinforcement Learning tasks. It comprises of:
* a finite **state space** $\mathcal{S}$
* a finite **action space** $\mathcal{A}$
* a **starting-state distribution** $\nu_0(s)$
* a **reward function** $r: \mathcal{S}\times\mathcal{A}\rightarrow [0,1]$ (or vector $r\in[0,1]^{|\mathcal{S}|\cdot|\mathcal{A}|}$)
* a **transition function** $p: \mathcal{S}\times\mathcal{A}\rightarrow \Delta_{\mathcal{S}}$ (or matrix $p \in\mathbb{R}^{|\mathcal{S}|\cdot|\mathcal{A}|\times|\mathcal{S}|}$)

<div>
<img src="https://drive.google.com/uc?export=view&id=11KyhHxuaileBEJ9fFZmIS5-GzncQgyqt" alt="agent-environment interaction" width="600"/>
</div>

### Minigrid Environment
Using the Minigrid (https://minigrid.farama.org/) package we develop a 2D grid world environment with explicit transition and reward functions.

The following code is meant to:
*   install configured minigrid package
*   import the minigrid package and other helpful packages
*   instantiate an $n$ by $n$ grid world environment with random seed to make the experiment fully reproducible.

In [ ]:
pip install "git+https://github.com/cipollone/rlss-practice1.git" --quiet

In [ ]:
from rlss_practice.environments import Room

import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import matplotlib as mpl

In [ ]:
n = 5
# seed = 4

# instantiate 5 by 5 grid with 0.1 failure probability
# env = Room(seed=seed, failure=0.1, size=n)
env = Room(failure=0.1, size=n)

#### Demo

In [ ]:
# some demo or exercise to get familiar with it

In [ ]:
# checkout env.StateT to get structure of states
# explain what each entry of the state tuple refers to
# checkout len(env.states) to see the number of states

In [ ]:
# actions are int between 0 and 2.
# describe what each action refers to
# checkout num of actions with env.action_space.n
# access actions with env.actions

In [ ]:
# checkout transition and reward
# run agent with random policy to see how it interacts with the environment

## Solving the Grid-world task

We would like to find a **stationary deterministic memoryless policy** or mapping from states to actions $\pi : \mathcal{S}\rightarrow \mathcal{A}$, that lets us arrive at the **goal state** sooner.

In other words, our objective to find $\pi$ which maximizes the **discounted return** from any starting state represented by:

\begin{align*}
    \rho(\pi) &= \mathbb{E}_{s_0\sim\nu_0, s_{t+1}\sim p(\cdot|s_t,\pi(s_t))}\left[\sum_{t=0}^\infty\gamma^t r(s_t,\pi(s_t))\right]\\
    &= \sum_{s}\nu_0(s) V^{\pi}(s)
\end{align*}

with the **discount factor** $\gamma\in[0,1)$.


&nbsp;

---

&nbsp;

Recall that the **state value function** for $s\in \mathcal{S}$ denoted

\begin{align*}
    V^\pi(s) &= \mathbb{E}_{s_{t+1}\sim p(\cdot|s_t,\pi(s_t))}\left[\sum_{t=0}^\infty\gamma^t r(s_t,\pi(s_t))\bigg|s_0=s\right]\\
    &=  \sum_{a}\pi(a|s)Q^{\pi}(s,a)
\end{align*}

represents the value of being in state $x$ and following policy $\pi$ while the **action-value function** for $x\in \mathcal{S}, a\in \mathcal{A}$ denoted

$$ Q^\pi(s,a) = r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V^{\pi}(s')$$

is the value of first taking action $a=\pi(s)$ in state $s$ then following policy $\pi$.

By standard notation, the **Bellman operator** of policy $\pi$ acting on functions $V : \mathcal{S}\rightarrow \mathbb{R}$ (or vectors $V\in\mathbb{R}^{|\mathcal{S}|}$) is given as:

$$(T^{\pi}V)(s) =  \sum_{a}\pi(a|s)\bigg[r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V(s')\bigg],\quad s\in\mathcal{S}$$


&nbsp;

---

&nbsp;

Ideally, we would like to find an **optimal policy** $\pi^*$ with:

$$ \pi^*(s) \in \underset{a\in\mathcal{A}}{\arg\max} \bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V^{\pi^*}(s')\bigg\},\quad x\in\mathcal{S}$$

that maximizes our immediate reward and future return.

The Bellman operator of $\pi^*$ (a.k.a the **Bellman optimality operator**) acting on functions $V : \mathcal{S}\rightarrow \mathbb{R}$ (or vectors $V\in\mathbb{R}^{|\mathcal{S}|}$) is given as:

$$(T^{*}V)(s) =  \max_{a}\bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V(s')\bigg\},\quad s\in\mathcal{S}$$


&nbsp;

---

&nbsp;

With the transition and reward function of the grid-world task available to us, we attempt to find an optimal policy via **Dynamic programming**. Precisely, we implement **Policy Iteration** and **Value iteration** methods introduced in the first lecture.

&nbsp;

### Policy Iteration (PI) Recap

**Idea**

Gradually advance to $\pi^{*}$ from an initial guess $\pi_0$ through a series of **policy evaluation** and **policy improvement** steps.

&nbsp;

**Policy Evaluation step**

Given a policy $\pi_k$, compute $V^{\pi_k}$ as

* $V^{\pi_k} = (I - \gamma p_{\pi_k})^{-1}r_{\pi_k}$

or estimate $V^{\pi_k}$ as $V_{k}$ with the following iterations

  * Initialize $V$ as $\mathbf{0}$ when $k = 0$ and $V_{k-1}$ otherwise, let $\pi = \pi_k$

    <div>
    <img src="https://drive.google.com/uc?export=view&id=1QaMg7a6HELjYycAnm6RE9vnzHD_fnaCn" alt="iterative policy evaluation" width="600"/>
    </div>

    Return $V_k = V$. <!--$(T^{\pi_{k}})^{n_{k}}V_{k-1}$-->

&nbsp;

**Policy Improvement step**

Obtain $\pi_{k+1}$ as the greedy policy w.r.t $V^{\pi_k}$ (or $V_{k}$). That is,
$$ \pi_{k+1}(s) \in \underset{a\in\mathcal{A}}{\arg\max} \bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{X}}p(s'|s,a)V^{\pi_{k}}(s')\bigg\},\quad s\in\mathcal{S}$$

&nbsp;

#### Putting everything together

Starting from an arbitrary stationary deterministic markovian policy $\pi_{0}$, for $k = 0,1,2,\cdots, K$ do:
* Compute $V^{\pi_k}$ or estimate $V^{\pi_k}$ with $V_{k}=(T^{\pi_{k}})^{n_{k}}V_{k-1}$
* Obtain $\pi_{k+1}$ as the greedy policy w.r.t $V^{\pi_k}$



&nbsp;

#### Theoretical guarantee

PI with "exact" policy evaluation steps finds an optimal policy after $K = \mathcal{\tilde{O}}((\mathcal{SA - S})/(1-\gamma))$ iterations.


&nbsp;

#### Notation

*   $r_{\pi}\in\mathbb{R}^{|\mathcal{S}|}$ so that for $s\in\mathcal{S}$, $r_{\pi}(s) = r(s,\pi(s))$.
*   $p_{\pi}\in\mathbb{R}^{|\mathcal{S}|\times|\mathcal{S}|}$ so that for $s,s'\in\mathcal{S}$, $p_{\pi}(s'|s) = p(s'|s,\pi(s))$.
*   $n_{k}$ is the number of loops required to compute $V_{k}$

In [ ]:
class PolicyIteration1:
    """
    Implements policy iteration with exact policy evaluation
    """
    def __init__(self,
                discount_factor: float,
                initial_policy = None,
                fig = None,
                ax = None) -> None:
      self.gamma = discount_factor
      self.policy = initial_policy

      if self.policy == None:
        np.random.seed(4)
        self.policy = {state: np.random.randint(0,env.action_space.n) for state in env.states}

      self.n_states = len(env.states)
      self.V = np.zeros((self.n_states,1))
      self.policy_stable = False
      self.max_value_gap = np.inf
      self.fig = fig
      self.ax = ax
      self.count = 0



    def evaluate_policy(self):
      """
      Given π_{k} compute V^{π_{k}} = (I - \gamma p_{π_{k}})^{-1}r_{π_{k}}.
      """

      plt.title('PI1: value estimate of π_'+str(self.count))

      self.V = np.linalg.inv(np.eye(self.n_states,self.n_states) - self.gamma*self.get_p_pi())
      self.V = self.V@self.get_r_pi()

      # ----- plot state values -----

      self.ax.bar(range(len(env.states)),
                  self.V.flatten(),
                  color = '#1f77b4')
      clear_output(wait=True)
      display(self.fig)
      self.count +=1
      return


    def get_policy(self):
      """
      Compute the greedy policy:
        π_{k+1}(s) = argmax_{a\in A} Q^{\pi_{k}}(s,a)
      where
        Q^{\pi_{k}}(s,a) = R(s,a) + gamma*<P(.|s,a),V^{\pi_{k}}>
      is the state-action value.
      """
      self.policy_stable = True

      for state in env.states:
        max_Q_value = self.get_expected_update(state, self.policy[state])

        for action in env.actions:
          Q_value = self.get_expected_update(state, action)

          if action != self.policy[state] and max_Q_value<Q_value:
            self.policy[state] = action
            max_Q_value = Q_value
            self.policy_stable = False
      return


    def get_expected_update(self, state, action):
      """
      Compute Bellman update at a state-action pair.

      input:
      state
      action
      discount factor (gamma)
      state values (state_values)

      output:
      r(s,a) + gamma <P(.|s,a),v>
      """
      value  = env.R[state][action]
      snext_index = 0

      for snext in env.states:
        value += self.gamma*env.T[state][action][snext]*self.V[snext_index]
        snext_index += 1

      return value


    def get_p_pi(self):
      """
      Given π_{k}, compute p_{π_{k}}
      """
      p_pi = np.zeros((self.n_states,self.n_states))
      s_index,snext_index = (0,0)

      for s in env.states:
        for snext in env.states:
          p_pi[s_index,snext_index] = env.T[s][self.policy[s]][snext]
          snext_index += 1
        snext_index=0
        s_index += 1

      return p_pi


    def get_r_pi(self):
      """
      Given π_{k}, compute r_{π_{k}}
      """
      r_pi = np.zeros((self.n_states,1))
      s_index = 0

      for s in env.states:
        r_pi[s_index][0] = env.R[s][self.policy[s]]
        s_index+=1

      return r_pi

In [ ]:
class PolicyIteration2:
    """
    Implements policy iteration with iterative policy evaluation
    """
    def __init__(self,
                discount_factor: float,
                theta: float,
                initial_policy = None,
                fig = None,
                ax = None) -> None:
      self.gamma = discount_factor
      self.theta = theta
      self.policy = initial_policy

      if self.policy == None:
        np.random.seed(4)
        self.policy = {state: np.random.randint(0,env.action_space.n) for state in env.states}

      self.V = {state: 0 for state in env.states}
      self.policy_stable = False
      self.max_value_gap = np.inf
      self.fig = fig
      self.ax = ax
      self.count = 0


    def evaluate_policy(self):
      """
      Starting from previous value estimate V_{k-1}
      estimate value of policy π_{k} by recursively applying
      the Bellman operator of the policy T^π_{k} to V_{k-1} until update is stable.
      """
      plt.title('PI2: value estimate of π_'+str(self.count))

      while self.max_value_gap > self.theta:
        self.max_value_gap = 0
        counter = 0

        for state in env.states:
          prev_statevalue = self.V[state]
          self.V[state] = self.get_expected_update(state, self.policy[state])
          self.max_value_gap = max(self.max_value_gap, np.abs(prev_statevalue - self.V[state]))

        # ----- plot state values -----
        if counter%10000==0:
          self.ax.bar(range(len(env.states)),
                      list(map(lambda state: self.V[state],env.states)),
                      color = '#DC143C')
          clear_output(wait=True)
          display(self.fig)
        counter+=1

      self.max_value_gap = np.inf
      self.count +=1
      return


    def get_policy(self):
      """
      Update policy to be the greedy policy:
        π(s) = argmax_{a\in A} Q(s,a)
      where
        Q(s,a) = R(s,a) + gamma*<P(.|s,a),v>
      is the state-action value.
      """
      self.policy_stable = True

      for state in env.states:
        max_Q_value = self.get_expected_update(state, self.policy[state])

        for action in env.actions:
          Q_value = self.get_expected_update(state, action)

          if action != self.policy[state] and max_Q_value<Q_value:
            self.policy[state] = action
            max_Q_value = Q_value
            self.policy_stable = False
      return


    def get_expected_update(self, state, action):
      """
      Compute Bellman update at a state-action pair.

      input:
      state
      action
      discount factor (gamma)
      state values (state_values)

      output:
      r(s,a) + gamma <P(.|s,a),v>
      """
      value  = env.R[state][action]

      for snext in env.states:
        value += self.gamma*env.T[state][action][snext]*self.V[snext]

      return value

### Value Iteration (VI) Recap

**Idea**

Gradually advance to $\pi^{*}$ with a single **iterative policy evaluation** step and **policy improvement**.

&nbsp;

**Policy Evaluation step**

No arbitrary starting policy needed. Estimate $V^{\pi^*}$ as follows:

  * For $k = 0,1,2,\cdots, K$

    * Set $V = \mathbf{0}$ when $k = 0$ and $V_{k-1}$ otherwise
      <div>
      <img src="https://drive.google.com/uc?export=view&id=1fsi3ZZgqZ-p061AxvSdeluWZTlwSjnGj" alt="iterative policy evaluation" width="400"/>
      </div>

      return $V$ as $V_{k}$.

&nbsp;

**Policy Improvement step**

Obtain $\hat{\pi}^{*}$ as the greedy policy w.r.t $V_{K}$. That is,

  $$ \hat{\pi}^{*}(s) \in \underset{a\in\mathcal{A}}{\arg\max} \bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{X}}p(s'|s,a)V_{K}(s')\bigg\},\quad s\in\mathcal{S}$$

&nbsp;

#### Putting everything together

* Estimate $V^{\pi^*}$ with $V_{K} = (T^*)^{K}\mathbf{0}$
* Obtain $\hat{\pi}^{*}$ as the greedy policy w.r.t $V_{K}$



&nbsp;

#### Theoretical guarantee

VI finds an **$\mathbf{\varepsilon}$-optimal policy** ($\pi^{\varepsilon}$) satisfying

$\qquad V^* - V^{\pi^{\varepsilon}}\leq \varepsilon e$

after $K = \mathcal{O}(\ln(2\gamma/\varepsilon(1-\gamma)^2)/(1-\gamma))$ iterations.


In [ ]:
class ValueIteration:
    """
    Implements value iteration
    """
    def __init__(self,
                discount_factor: float,
                epsilon: float,
                num_iterations = None,
                fig = None,
                ax = None) -> None:
      self.gamma = discount_factor
      self.K = num_iterations

      if self.K == None:
        self.K = math.ceil(np.log((2*self.gamma)/(epsilon*(1-self.gamma)**2))/(1-self.gamma))

      self.policy = {state: 0 for state in env.states}

      self.V = {state: 0 for state in env.states}
      self.fig = fig
      self.ax = ax


    def evaluate_policy(self):
      """
      Starting from an arbitrary value V_0(s) = 0 for s in S
      estimate the V^π* by recursively applying
      the Bellman optimality operator of the policy T^* to V_0 for K steps.
      """

      for k in range(self.K):
        plt.title('VI: value estimate of π* (V_'+str(k)+")")
        for state in env.states:
          self.V[state] = max(map(lambda action: self.get_expected_update(state, action), env.actions))

        # ----- plot state values -----

        if k%10==0:
          self.ax.bar(range(len(env.states)),
                      list(map(lambda state: self.V[state],env.states)),
                      color = '#008080')
          clear_output(wait=True)
          display(self.fig)
      return


    def get_policy(self):
      """
      Update policy to be the greedy policy:
        π(s) = argmax_{a\in A} Q(s,a)
      where
        Q(s,a) = R(s,a) + gamma*<P(.|s,a),v>
      is the state-action value.
      """
      for state in env.states:
        max_Q_value = self.get_expected_update(state, self.policy[state])
        for action in env.actions:
          Q_value = self.get_expected_update(state, action)
          if action != self.policy[state] and max_Q_value<Q_value:
            self.policy[state] = action
            max_Q_value = Q_value
      return


    def get_expected_update(self, state, action):
      """
      Compute Bellman update at a state-action pair.

      input:
      state
      action
      discount factor (gamma)
      state values (state_values)

      output:
      r(s,a) + gamma <P(.|s,a),v>
      """
      value  = env.R[state][action]

      for snext in env.states:
        value += self.gamma*env.T[state][action][snext]*self.V[snext]

      return value

### Run Policy and Value Iteration

In [ ]:
def run():
    discount_factor = 0.99
    epsilon = 0.01
    theta = 0.1

    mpl.style.use('seaborn-v0_8')


    #run policy iteration with exact policy evaluation step
    fig1 = plt.figure()
    ax1 = plt.gca()
    plt.ylabel("values")
    plt.xlabel("states")

    PI_planner1 = PolicyIteration1(discount_factor,
                                   fig = fig1,
                                   ax = ax1)

    while not PI_planner1.policy_stable:
      PI_planner1.evaluate_policy()
      PI_planner1.get_policy()

    #plt.close()


    #run policy iteration with iterative policy evaluation step
    fig2 = plt.figure()
    ax2 = plt.gca()
    plt.ylabel("values")
    plt.xlabel("states")

    PI_planner2 = PolicyIteration2(discount_factor,
                                   theta,
                                   fig = fig2,
                                   ax = ax2)

    while not PI_planner2.policy_stable:
      PI_planner2.evaluate_policy()
      PI_planner2.get_policy()

    #plt.close()


    # run value iteration
    fig3 = plt.figure()
    ax3 = plt.gca()
    plt.ylabel("values")
    plt.xlabel("states")

    VI_planner = ValueIteration(discount_factor,
                                epsilon,
                                fig = fig3,
                                ax = ax3)

    VI_planner.evaluate_policy()
    VI_planner.get_policy()

    #plt.close()

In [ ]:
run()

### Test Policy

Credit:

*   Bruno Scherrer, "a lecture on Markov Decision Processes and Dynamic Programming", June 2023, [link text](https://)
*   Csaba Szepesvári "a lecture series on Theoretical Foundations of Reinforcement Learning", 2020, [RL Theory course](https://rltheory.github.io/)
*   Richard S. Sutton, Andrew G. Barto "Reinforcement Learning: An Introduction", second edition, 2020, [link text](http://incompleteideas.net/book/RLbook2020.pdf)



